# TEP detailed analysis

In [ ]:
%matplotlib inline
%autosave 5

In [ ]:
# necessary modules -------------------------------------------------------------------------------------
import pyfesom2 as pf
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from netCDF4 import Dataset

import time
from datetime import date

#import sys
#sys.path.append('home/ollie/mozeisin/SkillMetrics')

import cartopy.crs as ccrs
import cartopy.feature as cfeature
#import pangaeapy.pandataset as pd

import warnings
warnings.simplefilter("ignore")

# own py_recom modules ----------------------------------------------------------------------
#!jupyter nbconvert --to=python plot_Taylor_normalized.ipynb

#from load_interp_WOA_python3 import WOAdata
from plot_Taylor_normalized import plt_Taylor_norm

In [ ]:
# After changing these parameters here, save the notebook and clear the output. 
# Then run this cell and the cells with the desired parameters.

# run specification -------------------------------------------------------------------------------------
runid      =  'fesom'
layerwise    = False
resultpath = '/work/ollie/mozeisin/results/f2r1.2/tep1' 
meshpath = '/work/ollie/mozeisin/mesh/mesh_fesom2.0/core2_meanz'

# period of analysis 
first_year = 1950
last_year  = 1980
    
years = np.arange(first_year,last_year+1,1)
years_last10 = np.arange(last_year-10,last_year+1,1)

# specification of analysis ------------------------------------------------------------------------------------
depth = 0

layerwise = False
mapproj = 'pc'
#Map projection. Options are Mercator (merc), Plate Carree (pc),
#North Polar Stereo (np), South Polar Stereo (sp),  Robinson (rob)
#Robinson projection is quite time consuming.

# export of analysis ------------------------------------------------------------------------------------
today = date.today().strftime("_%Y_%m_%d")

savepath = '/home/ollie/mozeisin/evaluation/mo_files/tep1/'
savefig = False
htmlname     =  'tep1' + today +'.html'
htmlpath = '/home/ollie/mozeisin/evaluation/mo_files/html/'
    

In [ ]:
# just to check
print('Processing years {4} to {5}\n\nReading out of {0}\nStoring graphs to {1}\nStoring html to {2} as {3}'.format(
    resultpath, savepath, htmlpath, htmlname,years[0],years[-1]))

print('\nLast ten years are {0}'.format(years_last10))

# Load mesh

In [ ]:
# mesh initialization -----------------------------------------------------------
mesh = pf.load_mesh(meshpath)

lon_fesom = mesh.x2
lat_fesom = mesh.y2

In [ ]:
def mygrid(m,grid=False):
    '''
    Draw cartopy natural earth feature to axis m
    '''
    #m.coastlines(resolution='110m', color='black', linewidth=1)
    land_poly = cfeature.NaturalEarthFeature('physical', 'land', '110m',edgecolor='k',facecolor=cfeature.COLORS['land'])
    m.add_feature(land_poly)

    if grid:
        g1 = m.gridlines(draw_labels = True)
        g1.xlabels_top = False
        g1.ylabels_right = False
        g1.xlabel_style = {'size': 16, 'color': 'gray'}
        g1.ylabel_style = {'size': 16, 'color': 'gray'}

# Chlorophyll

In [ ]:
#!jupyter nbconvert --to=python Chlsurf_OCCCI_Comparison_python3.ipynb
from Chlsurf_OCCCI_Comparison_python3 import Chlsurf_OCCCI_comp

# OCCCI   
matfileChlsurf           = '/work/ollie/projects/MarESys/evaluation/climatology_annual_chl_1deg_OCCCI_2012_2015.mat'

# analyze only last 10 years
Chl = Chlsurf_OCCCI_comp(runid,resultpath,savepath,meshpath,matfileChlsurf,
                          years_last10,
                          mapproj=mapproj,savefig=savefig)


# TEP

In [ ]:
NCfesom = resultpath + '/TEP.fesom.1948.nc'
!ncdump -h $NCfesom

In [ ]:
# various depths

units = 'TEP-C [mmol m$^{-3}$]'

depthi = 0
TEP_0 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_0='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depthi)

depthi = 5
TEP_1 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_1='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depthi)

depthi = 10
TEP_2 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_2='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depthi)

depthi = 20
TEP_3 = pf.get_data(resultpath, "TEP", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depthi)
label_3='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depthi)

In [ ]:
fig = pf.plot(mesh, [TEP_0,TEP_1,TEP_2,TEP_3], mapproj =  mapproj, units=units, 
              rowscol = (2,2), figsize = (15,10),
              levels = (0,15,61), #cmap = 'viridis',
              titles=[label_0,label_1,label_2,label_3])

if savefig:
    plt.savefig(savepath+'TEP_distribution_multidepths.png',dpi=300)

# integrals

In [ ]:
# load nodal area -------------------------------------------------------------------------------------
f = Dataset(resultpath + '/fesom.mesh.diag.nc', 'r')
nod_area = f.variables['nod_area'][:] # Moritz
nod_area_surface = nod_area[0,:]

#print('shape nod_area: {0}\nshape nod_area_surface: {1}'.format(np.shape(nod_area),np.shape(nod_area_surface)))
        
# calculating mean TEP per year -------------------------------------------------------------------------------
TEP00glob_total = np.zeros(len(years))
TEP05glob_total  = np.zeros(len(years))
TEP10glob_total = np.zeros(len(years))
#TEP20glob_total   = np.zeros(len(years))

TEP00ao_total = np.zeros(len(years))
TEP05ao_total = np.zeros(len(years))
TEP10ao_total = np.zeros(len(years))

TEP00so_total = np.zeros(len(years))
TEP05so_total = np.zeros(len(years))
TEP10so_total = np.zeros(len(years))


In [ ]:
## Arctic Oean, N of 70 -------------------------------------------------------------------------------
box_AO=[-180, 180, 70, 90] 
left, right, down, up = box_AO
mask_AO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_AO = nod_area[:,mask_AO]
#nod_area_AO_surface = nod_area[0,mask_AO]

## Souther Oean -------------------------------------------------------------------------------
box_SO =[-180, 180, -90, -50] 
left, right, down, up = box_SO
mask_SO = ((mesh.x2 >= left) & 
           (mesh.x2 <= right) & 
           (mesh.y2 >= down) & 
           (mesh.y2 <= up))

#nod_area_SO = nod_area[:,mask_SO]
#nod_area_SO_surface = nod_area[0,mask_SO]
#print(np.shape(nod_area_SO_surface))
        

In [ ]:
for i in range(len(years2)):
    ind_year = int(years2[i])
    #print('loading {0}'.format(ind_year))
    # TEP:units = "mmolC/m3"
    
    # 0m ############################
    TEP00glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 0,
                        compute=True, runid='fesom', silent=True)
    TEP00glob_total[i] = np.nanmean(TEP00glob)#*nod_area_surface)

    TEP00ao_total[i] = np.nanmean(TEP00glob[mask_AO])
    #TEP00ao_total[i] = np.nanmean(TEP00ao)
    
    TEP00so_total[i] = np.nanmean(TEP00glob[mask_SO])
    #TEP00so_total[i] = np.nanmean(TEP00so)
    
    # 5 m ############################
    TEP05glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 5,
                        compute=True, runid='fesom', silent=True)

    TEP05glob_total[i] = np.nanmean(TEP05glob)

    TEP05ao_total[i] = np.nanmean(TEP05glob[mask_AO])
    
    TEP05so_total[i] = np.nanmean(TEP05glob[mask_SO])

    # 10 m ############################
    TEP10glob = pf.get_data(resultpath, "TEP", ind_year, mesh, how="mean", 
                        depth= 10,
                        compute=True, runid='fesom', silent=True)

    TEP10glob_total[i] = np.nanmean(TEP10glob)

    TEP10ao_total[i] = np.nanmean(TEP10glob[mask_AO])
    
    TEP10so_total[i] = np.nanmean(TEP10glob[mask_SO])

In [ ]:
# plotting total TEP -------------------------------------------------------------------------------        
fig = plt.figure(figsize=(12,8), facecolor='w', edgecolor='k', tight_layout = True)
ylim = (0,5)

# 0 m ############################

plt.subplot(2, 3, 1)
plt.plot(years2,TEP00glob_total,'.-',label='0 m')
plt.plot(years2,TEP05glob_total,'.-',label='5 m')
plt.plot(years2,TEP10glob_total,'.-',label='10 m')
plt.title('Global mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 3, 2)
plt.plot(years2,TEP00ao_total,'.-',label='0 m')
plt.plot(years2,TEP05ao_total,'.-',label='5 m')
plt.plot(years2,TEP10ao_total,'.-',label='10 m')
plt.title('N of 70N mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

plt.subplot(2, 3, 3)
plt.plot(years2,TEP00so_total,'.-',label='0 m')
plt.plot(years2,TEP05so_total,'.-',label='5 m')
plt.plot(years2,TEP10so_total,'.-',label='10 m')
plt.title('S. Ocean mean TEP')
plt.ylabel(r'[mmol C m$^{-3}$]')
plt.ylim(ylim)
plt.legend(loc='upper right')

if savefig:
    plt.savefig(savepath+'TEP_timeseries.png',dpi=300)

# Interpolation to 1x1

In [ ]:
# intrpolation to 1x1
lat         = np.arange(-89.5,90.,1.)
lon         = np.arange(-179.5,180.,1.)
latdic, londic = np.meshgrid(lat, lon)

print(label_0)


TEP_0_interp = pf.fesom2regular(
            data = TEP_0,
            mesh = mesh,
            lons = londic, 
            lats = latdic)

In [ ]:
if True: # interpolation check            
    print('\nFESOM interpolation check:\noriginal min {0:5.4f} max {1:5.4f} mean {2:5.4f} \ninterpol min {3:5.4f} max {4:5.4f} mean {5:5.4f}'.format(
        np.nanmin(TEP_0),np.nanmax(TEP_0),np.nanmean(TEP_0),
        np.nanmin(TEP_0_interp),np.nanmax(TEP_0_interp),np.nanmean(TEP_0_interp)))
     

# North Polar Stereo

In [ ]:
# North Polar Stereo Projection
fig = plt.figure(figsize=(10,10))
ax1 = plt.subplot(projection = ccrs.NorthPolarStereo())

m1 = ax1.pcolormesh(londic, latdic, TEP_0_interp, 
    transform = ccrs.PlateCarree(),
    #norm=colors.LogNorm(vmin=np.nanmin(TEP_0_interp),vmax=np.nanmax(TEP_0_interp)),
    cmap='viridis')

#ax1.coastlines(resolution='110m', color='black', linewidth=1)
mygrid(ax1,grid=True)

plot_extent = [-180, 180, 70, 90]
ax1.set_extent(plot_extent, crs=ccrs.PlateCarree())

cbar = fig.colorbar(m1,#orientation = 'horizontal',
                    fraction=0.1, pad=0.1, shrink = 0.8) 
cbar.set_label('TEP-C [mmol C m$^{-3}$]', fontsize=20,)

plt.title('Interpolated '+label_0)

if savefig:
    plt.savefig(savepath+'TEP_interp_NPS.png', dpi=300)

# TEP in relation to CHl.a
figure y= log TEP XG eq , x= log Chla cmop. Zamanillo 2019 and Yamada 2015

### Convert TEP-C to TEP$_{color}$
TEP-C [$\mu$mol L$^{-1}$] = 0.05 TEP$_{color}$ [mg Xeq. L$^{-1}$]

In [ ]:
TEP_0_interp_mugXeqL = TEP_0_interp * 20 * 1000

### extracting regions

In [ ]:
## Arctic Oean, N of 70 
box_AO=[-180, 180, 70, 90] 
left, right, down, up = box_AO
mask_AO_1x1_lon = ((lon >= left) & (lon <= right))
mask_AO_1x1_lat = ((lat >= down) & (lat <= up))

if False:
    print(np.shape(mask_AO_1x1_lat))
    print(np.shape(mask_AO_1x1_lon))
    print(np.shape(TEP_0_interp))

if False: # control of cuting region
    fig = plt.figure(figsize=(10,10))
    ax1 = plt.subplot(projection = ccrs.NorthPolarStereo())

    m1 = plt.pcolormesh(londic[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
                        latdic[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
                        TEP_0_interp[mask_AO_1x1_lon][:,mask_AO_1x1_lat], 
        transform = ccrs.PlateCarree(),
        cmap='viridis')

    #ax1.coastlines(resolution='110m', color='black', linewidth=1)
    mygrid(ax1,grid=True)

    cbar = fig.colorbar(m1,#orientation = 'horizontal',
                        fraction=0.1, pad=0.1,shrink = 0.8) 
    cbar.set_label('TEP-C [mmol C m$^{-3}$]', fontsize=20)

    plt.title('Interpolated '+label_0)

In [ ]:
Chl_total_interp_AO = Chl.Chl_total_interp[:,mask_AO_1x1_lat].ravel()
TEP_0_interp_AO_mugXeqL = TEP_0_interp_mugXeqL[:,mask_AO_1x1_lat].ravel()

### set Chl.a minimum to 0.01 mg/m3

In [ ]:
def set_Chl_min(value):
    ind = np.where(Chl_total_interp_AO > value)
    Chl_total_interp_AO_min = Chl_total_interp_AO[ind]
    print('Number of Chl.a datapoints omitted: {0}'.format(len(Chl_total_interp_AO) - len(Chl_total_interp_AO_min) ))

    # apply to TEP as well
    TEP_0_interp_AO_min_mugXeqL = TEP_0_interp_AO_mugXeqL[ind]
    
    return [Chl_total_interp_AO_min,TEP_0_interp_AO_min_mugXeqL]

### plotting relation

In [ ]:
#x1 = np.log10(Chl_total_interp_AO_min)
#y2 = np.log10(TEP_0_interp_AO_min_mugXeqL)

x1, y1 = np.log10(set_Chl_min(0.01))

x2, y2 = np.log10(set_Chl_min(0.05))

In [ ]:
# linear regression
def get_res(x,y):
    res = stats.linregress(x,y)
    log = 'Regression: \nslope = %.2f intercept = %.2f, std error = %.3f\nR-squared = %.5f' % (
        res.slope, res.intercept, res.stderr, res.rvalue**2)
    print(log)
    return res, log

In [ ]:
plt.figure(figsize=(10,5))

plt.subplot(1,2,1)
plt.plot(x1,y1,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.01 mg m$^{-3}$')
plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x1,y1)
plt.plot(x1, res.intercept + res.slope*x1, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.subplot(1,2,2)
plt.plot(x2,y2,'.')
plt.ylim(0,6)
plt.xlim(-2.5,2.5)
plt.xlabel('log10 Chl.a [mg m$^{-3}$]\nmin(Chl.a) = 0.1 mg m$^{-3}$')
#plt.ylabel('log10 TEP [$\mu$g Xeq. L$^{-1}$]')
res, log = get_res(x2,y2)
plt.plot(x2, res.intercept + res.slope*x2, 'r', label='fitted line')
plt.text(-2,0.5,log)

plt.tight_layout()
if savefig:
    plt.savefig(savepath+'Chl_TEP_regression.png', dpi=300)


# Seasonal cycle

In [ ]:
TEP00_sc_raw = pf.get_data(resultpath, "TEP", years, mesh, depth = 0, how="ori", compute=False)
print(np.shape(TEP00_sc_raw))

months = np.arange(0,13,1)
depth_list = (0,5,10)

In [ ]:
# calculating mean TEP per month -------------------------------------------------------------------------------
TEP_glob_total_m = np.zeros((len(months),len(depth_list)))
#TEP05glob_total_m  = np.zeros((len(months),len(depth_list)))
#TEP10glob_total_m = np.zeros((len(months),len(depth_list)))

TEP_ao_total_m = np.zeros((len(months),len(depth_list)))
#TEP05ao_total_m = np.zeros((len(months),len(depth_list)))
#TEP10ao_total_m = np.zeros((len(months),len(depth_list)))

TEP_so_total_m = np.zeros((len(months),len(depth_list)))
#TEP05so_total_m = np.zeros((len(months),len(depth_list)))
#TEP10so_total_m = np.zeros((len(months),len(depth_list)))

In [ ]:
for i in range(len(years2)):
    ind_year = int(years2[i])
    print('loading {0}'.format(ind_year))
    # TEP:units = "mmolC/m3"
    
    # 0m ############################
    TEP_raw = pf.get_data(resultpath, "TEP", ind_year, mesh, how="ori", 
                        depth= 0,compute=False,  silent=True)
    
    TEP_glob_total_m[i] = np.nanmean(TEP_raw)

    ##TEP00ao_total[i] = np.nanmean(TEP00glob[mask_AO])
    
    #TEP00so_total[i] = np.nanmean(TEP00glob[mask_SO])
    
    # 5 m ############################
    

    # 10 m ############################
    

# Atlantic Transect

In [ ]:
lon_start = -60
lat_start = -80
lon_end = 10
lat_end = 83
npoints   = 50
lonlat = pf.transect_get_lonlat(lon_start, lat_start, lon_end, lat_end, npoints)

if True:
    pf.plot_transect_map(lonlat, mesh)

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)



In [ ]:
# get data
TEPall = pf.get_data(resultpath, "TEP", years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(TEPall))

print('TEP loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    last_year-10,last_year,depth, np.nanmin(TEPall), np.nanmax(TEPall), np.nanmean(TEPall)))

print('Plotting transect at lat {0} to {1}, lon {2} to {3}'.format(lat_start,lat_end,lon_start,lon_end))
#pf.plot_transect_map(lonlat, mesh)

# extract transect data
dist, tfTEP = pf.get_transect(TEPall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfTEP,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='TEP-C [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis')

# PCHO

In [ ]:
units = 'org C [mmol m$^{-3}$]'
label='Polysaccharides {0}-{1} at {2} m'.format(last_year-10,last_year,depth)

PCHO = pf.get_data(resultpath, "PCHO", years=years_last10, 
                   mesh = mesh, runid=runid, how="mean", compute=True, depth=depth)

fig = pf.plot(mesh, PCHO, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Atlantic transect 

# load data
PCHOall = pf.get_data(resultpath, "PCHO", years=years_last10, mesh=mesh, runid=runid, how="mean")#, depth=depth)

#print(np.shape(PCHO))

print('PCHO loaded for years {0}-{1}\nmin = {3}, max = {4}, mean = {5} [mmol/m3]'.format(
    years_last10[0],years_last10[-1],depth, np.nanmin(PCHOall), np.nanmax(PCHOall), np.nanmean(PCHOall)))

# extract transect data
dist, tfPCHO = pf.get_transect(PCHOall,mesh,lonlat)

pf.plot_xyz(mesh, 
            tfPCHO,
            dist,
            maxdepth=600,
            ncols=1,
            xlabel='Distance',
            label='PCHO [mmol/m3]',
            #levels=np.linspace(-2,2, 41),
            cmap='viridis',
            figsize=(15, 6))

# Intracellular Carbon

In [ ]:
# Diatom carbon
depth = 0
NCfesom = resultpath + '/DiaC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Diatom intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

DiaC = pf.get_data(resultpath, "DiaC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth=depth)
fig = pf.plot(mesh, DiaC, mapproj = mapproj, units=units, titles=label)

In [ ]:
# Phytplankton carbon

NCfesom = resultpath + '/PhyC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Phytoplankton intracell. Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'intracellular C [mmol C m$^{-3}$]'

PhyC = pf.get_data(resultpath, "PhyC", years=years_last10,
                   mesh=mesh, runid=runid, how="mean", depth = depth)

fig = pf.plot(mesh, PhyC, mapproj = mapproj, units=units, titles=label)

# Detritus Carbon

In [ ]:
NCfesom = resultpath + '/DetC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='DetC {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'Detritus C [mmol m$^{-3}$]'

DetC = pf.get_data(resultpath, "DetC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DetC, mapproj = mapproj, units=units, titles=label)

# extracellular carbon

In [ ]:
NCfesom = resultpath + '/EOC.fesom.1948.nc'
!ncdump -h $NCfesom

#label='TEP {0}-{1} at {2} m'.format(years_last10[0],years_last10[-1],depth)
#units = 'TEP-C [mmol m$^{-3}$]'
#TEP = pf.get_data(resultpath, "EOC", years_last10, mesh, runid=runid, how="mean", compute=True, depth=depth)
#fig = pf.plot(mesh, TEP, mapproj = mapproj, units=units, titles=label)

# NPP

In [ ]:
# map of total Diatom NPP 

NCfesom = resultpath + '/NPPd.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mmolC/m2/d to mg C/m2/d

label='Diatom NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPd = pf.get_data(resultpath, "NPPd", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True, silent=True)

#print(np.shape(NPPd))
# NPP is already vertically integrated!

NPPd = NPPd * 12.01
fig = pf.plot(mesh, NPPd, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# map of nanophytoplankton NPP

NCfesom = resultpath + '/NPPn.fesom.1948.nc'
#!ncdump -h $NCfesom

# 12.01* data --> mg C/m2/d

label='Nanophytoplankton NPP {0}-{1}'.format(last_year-10,last_year)
units = 'Mean NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPn = pf.get_data(resultpath, "NPPn", 
                   years_last10,
                   mesh, runid=runid, how="mean", compute=True,silent=True)

#print(np.shape(NPPn))
# NPP is already vertically integrated!

NPPn = NPPn *12.01

fig = pf.plot(mesh, NPPn, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

In [ ]:
# total NPP
label='Total Net Primary Production {0}-{1}'.format(last_year-10,last_year)
units = 'Mean total NPP [mg C m$^{-2}$ d$^{-1}$]'

NPPtot = NPPd + NPPn
fig = pf.plot(mesh, NPPtot, mapproj = mapproj, units=units, titles=label, 
              levels = (0,1100,56),
              cmap='viridis')

# DIC

In [ ]:
# DIC
depth = 0
NCfesom = resultpath + '/DIC.fesom.1948.nc'
#!ncdump -h $NCfesom

label='Dissolved inorganic Carbon {0}-{1} at {2} m'.format(last_year-10,last_year,depth)
units = 'DIC [mmol  m$^{-3}$]'
DIC = pf.get_data(resultpath, "DIC", years_last10,mesh, how="mean", compute=True, depth=depth)
fig = pf.plot(mesh, DIC, mapproj = mapproj, units=units, titles=label)


# export to html

In [ ]:
#!jupyter nbconvert TEP_detailed_analysis.ipynb --output-dir=$htmlpath --output $htmlname --to html --TemplateExporter.exclude_input=True